In [25]:
import ee
import geemap

In [26]:
ee.Initialize()

In [59]:
Map = geemap.Map()

In [40]:
# mask of pixels that were unchanged (until 2015)
MakMarco = ee.Image("users/marcogirardello/phenoutils/mask_unchanged_500m")

In [50]:
# add NDVI to data
def addNDVI(image):
    image = image.updateMask(MakMarco.eq(1))
    return image.addBands(image.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')).float()

# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)

# mask out low quality pixels (based on flags)
def maskPixels(image0):
    #Select the QA band
    QA = image0.select('StateQA')
    # Get the land_water_flag bits
    landWaterFlag = getQABits(QA, 3, 5, 'land_water_flag')
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
    # Get the cloud_shadow bit
    cloudShadows = getQABits(QA, 2, 2, 'cloud_shadow')
    # Get the Pixel is adjacent to cloud bit
    cloudAdjacent = getQABits(QA, 13, 13, 'cloud_adj')
    # Get the internal cloud flag
    cloud2 = getQABits(QA, 10, 10, 'cloud_internal')
    # Get the internal fire flag
    fire = getQABits(QA, 11, 11, 'fire_internal')
    # Get the MOD35 snow/ice flag
    snow1 = getQABits(QA, 12, 12, 'snow_MOD35')
    # Get the internal snow flag
    snow2 = getQABits(QA, 15, 15, 'snow_internal')
    # create mask
    mask = landWaterFlag.eq(1).And(cloud.Not()).And(cloudShadows.Not()).And(cloudAdjacent.Not()).And(cloud2.Not()).And(fire.Not()).And(snow1.Not()).And(snow2.Not())
    return image0.updateMask(mask) 
            

In [51]:
dataset = ee.ImageCollection('MODIS/006/MOD09A1').filter(ee.Filter.date('2018-01-01', '2018-05-01'))

In [54]:
# mask pixels on the basis of quality flags
MOD09masked = dataset.map(maskPixels)
# add NDVI band
MOD09ndvi = MOD09masked.map(addNDVI).select(['NDVI'])

In [55]:
img = MOD09ndvi.first()

In [60]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

80

In [62]:
Map.addLayer(img)

In [7]:
def qamask(x):
    Q = x.select(['DetailedQA'])
    nieve = Q.bitwiseAnd(ee.Image.constant(32768))
    tierra = Q.bitwiseAnd(ee.Image.constant(16384))
    nube = Q.bitwiseAnd(ee.Image.constant(1024))
    aerosol = Q.bitwiseAnd(ee.Image.constant(192))
    filtro = nieve.add(tierra).add(nube).add(aerosol) 
    filtro.lte(64)

In [8]:
# function that joins imagecollections 
def joincoll(leftCollection, rightCollection, newproperty):
    filter = ee.Filter.equals({
    leftField: 'system:time_start', 
    rightField: 'system:time_start'})
    